# Nonlinear Elasticity in a multipatch domain


In [ ]:
%pylab inline
import scipy
import itertools

from pyiga import bspline, assemble, vform, geometry, vis, solvers

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

# assemble matrix
from scipy.sparse import coo_matrix, block_diag, bmat
from scipy.sparse import bsr_matrix
#inverse
from scipy.sparse.linalg import inv

# line search
from scipy.optimize import line_search


In [ ]:
p = 3 #3  # spline degree 
x_el= 40
y_el= 8
n_el = (x_el, y_el)  #(40,15)# number of elements in y/x direction, isotropic material
# displacement space: degree p,   continuity p-1
multi= 2
kvs_u = tuple(bspline.make_knots(p, 0.0, 1.0, n, mult=multi) for n in n_el) # or : mult=2
m_u = tuple(kv.numdofs for kv in kvs_u)

In [ ]:
# rubber, Lamé-coeff.
Mu = 3 # [MPa]
Lam = 0.28416 #  [MPa] (under ass. ny=0.48, almost incompressible)
# incompressible: Lam = 0


In [ ]:
# define geometry
# realistic model of an artery: r_inner= 4mm, r_outer= 7mm (preprint)
# human carotid artery: r_i= 3.1 mm, r_o= 4mm (Holzapfel)
r_out = 4
r_in = 3.1

geos = [
 
    geometry.quarter_annulus(r1=r_in, r2=r_out),
    geometry.quarter_annulus(r1=r_in, r2=r_out).rotate_2d(3*pi/2),
    geometry.quarter_annulus(r1=r_in, r2=r_out).rotate_2d(pi),
    geometry.quarter_annulus(r1=r_in, r2=r_out).rotate_2d(pi/2)
]

patches_u = [(kvs_u, g) for g in geos]

# Here we auto-detect the interfaces between the patches.
# It is also possible to join them manually using
# MP.join_boundaries(...) followed by MP.finalize().
MP_u = assemble.Multipatch(patches_u, automatch=True)

In [ ]:
figsize(8,6)
for g in geos:
    vis.plot_geo(g, gridx=np.linspace(0, 1, x_el), gridy=np.linspace(0, 1, y_el))
axis('scaled');

In [ ]:
class multipatch_block_handler:
    
    # Takes array of multi-patch objects
    def __init__( self, multi_patches ):
        self.multi_patches = multi_patches
        self.numpatches = multi_patches[0].numpatches
        for multi_patch in multi_patches:
            if multi_patch.numpatches != self.numpatches:
                print("Inconsistent numbers of patches")

    def patch_to_global(self, p):
        first = True
        for multi_patch in self.multi_patches:
            if first:
                X = multi_patch.patch_to_global(p)
                first = False
            else:
                X = scipy.sparse.block_diag((X,multi_patch.patch_to_global(p)))
        return X
    
    def compute_dirichlet_bcs(self, data):
        first = True
        p = 0
        offset = 0
        for multi_patch in self.multi_patches:
            data_p = []
            for item in data:
                #print(item[3])
                if len(item)<4 or p in item[3]:
                    data_p.append( (item[0], item[1], lambda *x : item[2](*x)[p]) )
            if len(data_p)>0:
                bcs_p = multi_patch.compute_dirichlet_bcs(data_p)
                #print('bcs_p', bcs_p)
                if first:
                    bcs = list(bcs_p)
                    first = False
                else:
                    #Indices need offset
                    bcs[0] = numpy.concatenate((bcs[0],bcs_p[0]+offset))
                    #Values are kept as-is
                    bcs[1] = numpy.concatenate((bcs[1],bcs_p[1]))
            offset += multi_patch.numdofs
            p += 1
        return tuple(bcs)
    
    def compute_local_offset_for_component(self, p, c):
        offset = 0
        for cc in range(c):
            dim = 1
            kvs, geo = self.multi_patches[cc].patches[p]
            for kv in kvs:
                dim *= kv.numdofs
            offset += dim
        return offset
    

In [ ]:
# Multipatch objects for all variables
MP_block = multipatch_block_handler( [MP_u, MP_u] )

In [ ]:
# source term f 
def f(x, y): return (0.0, 0.0)

#Neumann BC
#def gN(x, y): return (x, -y)  #outer pressure
def gN(x, y): 
    return (x/r_in*loading_value, y/r_in*loading_value)  #inner  pressure, normalized vector


In [ ]:
# define Dirichlet boundary function 
def g_zero(x, y): return (0.0, 0.0)

# set up Dirichlet boundary conditions
bc = MP_block.compute_dirichlet_bcs([
    (1, 'right', g_zero ) 
])


In [ ]:
# define constant spline functions for integration
kvs_j = tuple(bspline.make_knots(0, 0.0, 1.0, n, mult=multi) for n in n_el) # constant basis vector for integration

In [ ]:
def plot_grid(x, y, ax=None, **kwargs):
    """Plot a grid over a geometry"""
    ax = ax or plt.gca()
    segs1 = np.stack((x, y), axis=2)
    segs2 = segs1.transpose(1, 0, 2)
    ax.add_collection(LineCollection(segs1, **kwargs))
    ax.add_collection(LineCollection(segs2, **kwargs))
    ax.autoscale()

In [ ]:
#split the solution into its components (displacement in x- and y- direction)
# visualization per patch
def get_defplot(u, patches_u, kvs_u):
    """Split solution vector into displacement components."""
    u1 = u[:MP_u.numdofs] 
    u2 = u[MP_u.numdofs:2*MP_u.numdofs]
   
    # restrict solution to each individual patch - BSpline functions
    u1_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u1)
           for p in range(len(patches_u))]
    u2_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u2)
           for p in range(len(patches_u))]

    # evaluate displacement (and "pressure") over a grid in the parameter domain
    # grid variables
    #ref = y_el # take number of elements in y-dir  #OR: set manually!
    xgrid=np.linspace(0, 1, x_el)
    ygrid=np.linspace(0, 1, y_el)
    #xgrid = linspace(0, 1, ref)
    xygrid = (xgrid, ygrid)
    #len_xgrid= len(xgrid)
    vrange= None
   
    figsize(10,10)
    fig, ax = plt.subplots()
    
    # visualization per patch
    for (u1_func, u2_func, (kvs, geo)) in zip(u1_funcs, u2_funcs, patches_u): #u_funcs 
        dis1 = u1_func.grid_eval(xygrid) #x-value
        dis2 = u2_func.grid_eval(xygrid) #y-value
        dis = np.stack((dis1,dis2), axis=-1)
        G = geo.grid_eval(xygrid)
        plot_grid(G[..., 0], G[..., 1], ax=ax, color="lightgrey")
        plot_grid(G[..., 0] + dis[..., 0], G[..., 1] + dis[..., 1], ax=ax, color="black")
        
        C = np.sqrt(np.power(dis[..., 0], 2) + np.power(dis[..., 1], 2))
        if vrange is None:
            vrange = (C.min(), C.max())
            
        plt.pcolormesh(G[..., 0] + dis[..., 0], G[..., 1] + dis[..., 1], C, shading='gouraud', cmap='viridis', 
                                    vmin=vrange[0], vmax=vrange[1])
        #vis.plot_deformation(dis, ref, geo, ax, vmin=0.0, vmax=1.5e-3)
    colorbar();
    axis('equal')


In [ ]:
# Compute solution of nonlinear elasticity problem

In [ ]:
#split the solution into its components (displacement in x- and y- direction) and convert to BSpline function
def get_components(u, kvs_u):
    """Split solution vector into displacement components."""
    N = np.prod(tuple(kv.numdofs for kv in kvs_u))
    assert u.shape[0] == 2*N
    u1 = u[:N].reshape(m_u)          
    u2 = u[N:].reshape(m_u)    
    U = np.stack((u1,u2), axis=-1)
    return bspline.BSplineFunc(kvs_u, U)

In [ ]:
def get_components_u(u, kvs_u):
    """Split solution vector into displacement components."""
    N = np.prod(tuple(kv.numdofs for kv in kvs_u))
    u1 = u[:N].reshape(m_u)          
    u2 = u[N:].reshape(m_u)    
    return np.stack((u1,u2), axis=-1)

In [ ]:
# Energy functional
def energy(u_p, kvs_ux, geo_ux, X):
    dis = get_components(X.T @ u_p, kvs_ux)
    e_term = '(.5*( grad(dis).T + grad(dis) + grad(dis).dot(grad(dis).T) ))'
    return sum(assemble.assemble(f'( Lam/2*tr({e_term})*tr({e_term}) + Mu*tr({e_term}.dot({e_term})) ) * aux * dx',
                             kvs_ux, bfuns=[('aux',1)], geo=geo_ux, dis=dis, Lam=Lam, Mu=Mu, idmat=np.identity(2)).ravel())

#def nonlinear_form_old(u_p, kvs_ux, geo_ux, X):
#    dis = get_components(X.T @ u_p, kvs_ux)
#    
#    z_lam = Lam/2* assemble.assemble('(tr( grad(dis).T + grad(dis) + grad(dis).T @ grad(dis)) * div(v) + inner(grad(dis) * tr( grad(dis).T + grad(dis) + grad(dis).T @ grad(dis)) , grad(v)) )*dx'
#                                     , kvs_ux, bfuns=[('v',2)], geo=geo_ux, dis=dis)
#    z_mu = Mu* assemble.assemble('inner(grad(dis).T + grad(dis) + grad(dis).T @ grad(dis), grad(v))*dx + inner(grad(dis)@(grad(dis).T + grad(dis) + grad(dis).T@grad(dis)), grad(v)) *dx'
#                                    , kvs_ux, bfuns=[('v',2)], geo=geo_ux, dis=dis)
#    return (z_lam + z_mu).ravel()  # vector z


# Nonlinear variational form
def nonlinear_form(u_p, kvs_ux, geo_ux, X):
    dis = get_components(X.T @ u_p, kvs_ux)
    e_term = '(.5*( grad(dis).T + grad(dis) + grad(dis).dot(grad(dis).T) ))'
    f_term = '( idmat + grad(dis) )'
    return assemble.assemble(f'inner( ( Lam*tr({e_term})*idmat + 2*Mu* {e_term} ).dot({f_term}), grad(v) ) * dx',
                             kvs_ux, bfuns=[('v',2)], geo=geo_ux, dis=dis, Lam=Lam, Mu=Mu, idmat=np.identity(2)).ravel()

# linearized variational form
#def linearized_form_old(u_p, kvs_ux, geo_ux, X):
#    dis = get_components(X.T @ u_p, kvs_ux)
# 
#    #A_lin_lam = Lam/2* assemble.assemble('(inner( tr( grad(dis).T + grad(dis) + grad(dis).T @ grad(dis)) * grad(u), grad(v)) + tr( grad(u) + grad(dis).T @ grad(u) ) * div(v) + inner( tr(grad(u) + grad(dis).T @ grad(u)) * grad(dis), grad(v) ) ) * dx'
#    A_lin_lam = Lam/2* assemble.assemble('(inner( tr( grad(dis).T + grad(dis) + grad(dis).T @ grad(dis)) * grad(u), grad(v)) + 2* tr( grad(u) + grad(dis).T @ grad(u) ) * div(v) + inner( tr(grad(u) + grad(dis).T @ grad(u)) * grad(dis), grad(v) ) ) * dx'
#                                       , kvs_ux, bfuns=[('u',2), ('v',2)], geo=geo_ux, dis=dis) #matrix A_lam
#    A_lin_mu = Mu * assemble.assemble('(inner( grad(u)@(grad(dis) + grad(dis).T + grad(dis).T @ grad(dis)) +  (grad(u) + grad(u).T + grad(dis).T @ grad(u) + grad(u).T @ grad(dis)) + grad(dis)@(grad(u)+grad(u).T + grad(dis).T @ grad(u) + grad(u).T @ grad(dis)), grad(v))) * dx'
#                              , kvs_ux, bfuns=[('u',2), ('v',2)], geo=geo_ux, dis=dis) #matrix A_mu
#    
#    return A_lin_lam + A_lin_mu # matrix A

# Linearized variational form
def linearized_form(u_p, kvs_ux, geo_ux, X):
    dis = get_components(X.T @ u_p, kvs_ux)
    e_term = '(.5*( grad(dis).T + grad(dis) + grad(dis).T.dot(grad(dis)) ))'
    e_deriv = '(.5*( grad(u).T + grad(u) + grad(u).T.dot(grad(dis)) + grad(dis).T.dot(grad(u)) ))'
    f_term = '( idmat + grad(dis) )'
    f_deriv = 'grad(u)'
    return assemble.assemble(f'inner( ( Lam * tr({e_term}) * idmat + 2*Mu*{e_term} ).dot({f_deriv}) + ( Lam * tr({e_deriv}) * idmat + 2*Mu*{e_deriv} ).dot({f_term}), grad(v) ) * dx',
                               kvs_ux, bfuns=[('u',2),('v',2)], geo=geo_ux, dis=dis, Lam=Lam, Mu=Mu, idmat=np.identity(2))


In [ ]:
#derive energy functional (with Neumann + Robin)
def ass_energy(u): 
    j = 0
    for p in range(MP_block.numpatches): # go through each patch
        X = MP_block.patch_to_global(p)
        kvs_ux, geo_ux = MP_block.multi_patches[0].patches[p]
        j_p = energy(u, kvs_ux, geo_ux, X)
        dis = get_components(X.T @ u, kvs_ux)
        
        j_R = 0  ###!!
        j_N = 0  ###!!
    
        # Robin-Data
        for item in robin_data:
            if item[0]==p:
                R_p = item[3]/2 * assemble.assemble('inner(dis,dis) * aux * ds', kvs_j, bfuns=[('aux',1)], dis=dis, geo=geo_ux, boundary=item[1], layout='packed') #first attempt 
                j_R += R_p.sum()
        
        for item in neu_data:
            if item[0]==p:
                N_e = -loading_value*assemble.assemble('inner(n, dis) * aux * ds', kvs_j, bfuns=[('aux',1)], geo=geo_ux, boundary=item[1], dis=dis)
                #print('N_e1=', N_e1)
                #N_e  = assemble.assemble('inner(g,dis) *v *ds', kvs_j, bfuns=[('v',1)], geo=geo_ux, g=item[2], boundary=item[1], dis=dis) 
                #print('N_e=', N_e)
                j_N += N_e.sum()

        j += (j_p + j_R - j_N)
        
    return j

#assemble linear system and right-hand side (with Neumann + Robin)
def ass_nonlinsystem_RN(u):
    first = True
    firstR = True
    
    for p in range(MP_block.numpatches):
        X = MP_block.patch_to_global(p)
        
        # All the geometries are supposed to be equal; also kvs_ux and kvs_uy are equal
        kvs_ux, geo_ux = MP_block.multi_patches[0].patches[p] #kvs_uy, geo_uy
        
        # The whole patch-local stiffness matrix
        A_p = linearized_form(u, kvs_ux, geo_ux, X) # take linearized variatonal form
        
        # The patch-local Neumann boundary data, right-hand side
        #b_p = assemble.inner_products(kvs_ux, f, f_physical=True, geo=geo_ux).ravel() - nonlinear_form(u, kvs_ux, geo_ux, X)  # for arbitrary rhs
        b_p = -nonlinear_form(u, kvs_ux, geo_ux, X)
      
        # Robin-Data
        #dis = get_components(X.T @ u, kvs_ux)
        for item in robin_data:
            if item[0]==p:
                AR_u = item[3] * assemble.assemble('u * v * ds', kvs_ux, bfuns=[('u',1),('v',1)], geo=geo_ux, #Here, only scalar!
                                             format='csr', layout='blocked',boundary=item[1])

                bdofs_R = assemble.boundary_dofs(kvs_ux, item[1], ravel=True)
                for c in range(2): #x and y 
                    offset = MP_block.compute_local_offset_for_component(p,c)
                    if firstR:
                        AR = X.tocsr()[:,bdofs_R+offset] @ AR_u @ X.tocsr()[:,bdofs_R+offset].T
                        firstR = False
                    else:
                        AR += X.tocsr()[:,bdofs_R+offset] @ AR_u @ X.tocsr()[:,bdofs_R+offset].T
                     

        for item in neu_data:
            if item[0]==p:
                #N_en = assemble.assemble('inner(g,v)*ds', kvs_ux, bfuns=[('v',2)], geo=geo_ux, g=item[2], boundary=item[1], layout='packed') #first attempt 
                #print('N_e', np.linalg.norm(N_e))
            
                # normal vector with loading
                N_en = -loading_value * assemble.assemble('inner(n,v)*ds', kvs_ux, bfuns=[('v',2)], geo=geo_ux, boundary=item[1], layout='packed')
                #print('N_en2=',N_en2)
                #print('N_en', np.linalg.norm(N_en))
                
                bdofs = assemble.boundary_dofs(kvs_ux, item[1], ravel=True)
                for c in range(2): #x and y 
                    offset = MP_block.compute_local_offset_for_component(p,c)
                    for i in range(len(bdofs)):
                        b_p[bdofs[i]+offset] += N_en[i, 0, c] # (43,1,2)

        if first:
            A = X @ A_p @ X.T
            b = X @ b_p 
            first = False
        else:
            A += X @ A_p @ X.T
            b += X @ b_p 
    
    # b is a residual of the form l-A(u)
    
    return A+AR, b-AR@u


In [ ]:
def J(u):
    return ass_energy(u)

def grad_J(u):
    A, b = ass_nonlinsystem_RN(u)
    return b

# TODO: It is inefficient to:
#           (i)   compute A if only grad_J is used,
#           (ii)  compute A and/or b repeatedly for the same u,
#           (iii) recompute the vector from the Neumann conditions for rhs (term needs re-scaling for changes of scaling),
#           (iv)  recompute the Robin matrix.
# Further impovement possible by using "updatable" field as in the solve-navier-stokes example.

In [ ]:
def ass_mass():
    first = True
    
    for p in range(MP_block.numpatches):
        X = MP_block.patch_to_global(p)
        
        # All the geometries are supposed to be equal; also kvs_ux and kvs_uy are equal
        kvs_ux, geo_ux = MP_block.multi_patches[0].patches[p] #kvs_uy, geo_uy

        M_x = assemble.mass(kvs_ux, geo_ux)
        M_p = scipy.sparse.bmat(
            [[M_x, None],
            [None, M_x]], format='csr')
        
        if first:
            M = X @ M_p @ X.T
            first = False
        else:
            M += X @ M_p @ X.T
           
    return M

In [ ]:
def Wolfe_LineSearch(f, f_grad, xk, pk):
    # Wolfe-Powell_Line search
    print('Start line search')
    
    mu1 = 0.01
    sigma = 0.9
    tau = 0.1 
    tau1 = 0.3
    tau2 = 0.9
    gsi1 = 1
    gsi2 = 10
    
    def phi(alpha):
        return f(xk + alpha * pk)

    def derphi(alpha):
        #print('f_grad=', f_grad(xk + alpha * pk))
        return np.dot(f_grad(xk + alpha * pk), pk)
    
    alpha_L = 0
    alpha_R = None
    phi_L = phi(0)
    derphi0 = derphi(0)
    derphi_L = derphi0
    
    alphainf= True
    alpha =1 
    phi_min = -1.e2
    
    eps0 = 1.e-6
    max_iter = 5
    n_iter = 0
    #alpha_int= None
    
    
    if derphi0 >= 0:
        print('  no descent direction: {}'.format(derphi_L))
        return None
    
    while n_iter < max_iter:
        print("  {}. alpha between {} and {}".format(n_iter,alpha_L,alpha_R))
        n_iter +=1
        phi_hat= phi(alpha)
        derphi_hat = derphi(alpha)
        
        if alpha > 1: # alpha too big # alpha in(0,1)?
            print('alpha >1')
            alpha_R = alpha
            alpha= alpha_L + tau1*(alpha_R - alpha_L)
            if alpha_L != 0  and  alpha_R <= alpha_L*1.5:
                return alpha_L
            
        else:
            #phi_hat= phi(alpha)
            #print('     alpha = {} gives {}'.format(alpha,math.sqrt(phi_hat)))
            if phi_hat < phi_min: # function unbounded to the bottom
                return alpha 

            if phi_hat > phi(0) + mu1*alpha*derphi(0): #alpha too big # 
                print('     alpha too big')
                alphainf= False
                alpha_R = alpha
                delta = alpha_R - alpha_L
               # print('  delta=', delta)
                ## interpolation
                c = (phi_hat - phi_L - derphi_L*delta)/delta**2
                alpha_tilde = alpha_L - derphi_L/(2*c)
                alpha = min(max(alpha_L+ tau*delta, alpha_tilde), alpha_R - tau*delta)
                print('alpha_int=', alpha)
                #alpha_int = alpha

            else: # alpha not too big
                #derphi_hat = derphi(alpha)
                #print('     derphi_hat = {} and sigma*derphi(0) = {}'.format(derphi_hat,sigma*derphi(0)))
                if derphi_hat < sigma*derphi0: # alpha too small
                    print('     alpha too small')
                    if alpha_R != None and .6 * alpha_R < alpha:
                        print('     Interval small enough, so dont bother.')
                        return alpha
                    if alpha_R == None: # alpha_R == inf
                        if derphi_L/derphi_hat > (1+ gsi2)/gsi2:
                            alpha_tilde = alpha + (alpha - alpha_L)*max(derphi_hat/(derphi_L -derphi_hat), gsi1)
                        else:
                            alpha_tilde = alpha + gsi2*(alpha - alpha_L)
                    else: # alpha_R < inf
                        if derphi_L/derphi_hat > 1 + (alpha-alpha_L)/(tau2*(alpha_R -alpha)):
                            alpha_tilde = alpha + max((alpha-alpha_L)*derphi_hat/(derphi_L - derphi_hat), tau1*(alpha_R- alpha))
                        else:
                            alpha_tilde = alpha + tau2*(alpha_R - alpha)
                    ## update
                    alpha_L = alpha
                    phi_L = phi_hat
                    derphi_L = derphi_hat
                    alpha = alpha_tilde
                else:
                    print('     alpha neither too big, nor too small')
                    return alpha

    #return alpha


In [ ]:
def Wolfe_LineSearch_st(f, f_grad, xk, pk, alpha_R=None):
    # Wolfe-Powell_Line search
    print('Start Wolfe line search')
    
    mu1 = 0.01
    sigma = 0.9
    tau = 0.1
    tau1 = 0.3
    tau2 = 0.9
    gsi1 = 1
    gsi2 = 10
    
    def phi(alpha):
        return f(xk + alpha * pk)

    def derphi(alpha):
        return np.dot(f_grad(xk + alpha * pk), pk)
    
    alpha_L = 0
    #alpha_R = 1 #None
    phi0 = phi(0)
    phi_L = phi0
    derphi0 = derphi(0)
    derphi_L = derphi0
    alpha = 1 
    phi_min = -1.e2
    
    eps0 = 1.e-6
    max_iter = 20
    n_iter = 0
    
    if derphi0 >= 0:
        print('  no descent direction: {}'.format(derphi_L))
        return None
    
    while n_iter < max_iter:
        print("  {}. alpha between {} and {}".format(n_iter,alpha_L,alpha_R))
        n_iter +=1

        phi_hat = phi(alpha)
        #if phi_hat < phi_min: # function unbounded to the bottom
        #    return alpha 

        print('     alpha = {} gives {}'.format(alpha,math.sqrt(phi_hat)))
        
        if phi_hat > phi0 + mu1*alpha*derphi0: # alpha too big
            print('     alpha too big')
            alpha_R = alpha
            delta = alpha_R - alpha_L
            ## interpolation
            c = (phi_hat - phi_L - derphi_L*delta)/delta**2
            alpha_tilde = alpha_L - derphi_L/(2*c)
            alpha = min(max(alpha_L + tau*delta, alpha_tilde), alpha_R - tau*delta)

        else: # alpha not too big
            derphi_hat = derphi(alpha)
            #print('     derphi_hat = {} and sigma*derphi(0) = {}'.format(derphi_hat,sigma*derphi(0)))
            if derphi_hat < sigma*derphi0: # alpha too small
                print('     alpha too small')
                if alpha_R != None and .6 * alpha_R < alpha:
                    print('     Interval small enough, so dont bother.')
                    return alpha
                if alpha_R == None: # alpha_R == inf
                    if derphi_L/derphi_hat > (1+ gsi2)/gsi2:
                        alpha_tilde = alpha + (alpha - alpha_L)*max(derphi_hat/(derphi_L -derphi_hat), gsi1)
                    else:
                        alpha_tilde = alpha + gsi2*(alpha - alpha_L)
                else: # alpha_R < inf
                    if derphi_L/derphi_hat > 1 + (alpha-alpha_L)/(tau2*(alpha_R -alpha)):
                        alpha_tilde = alpha + max((alpha-alpha_L)*derphi_hat/(derphi_L - derphi_hat), tau1*(alpha_R- alpha))
                    else:
                        alpha_tilde = alpha + tau2*(alpha_R - alpha)
                ## update
                alpha_L = alpha
                phi_L = phi_hat
                derphi_L = derphi_hat
                alpha = alpha_tilde
            else:
                print('     alpha neither too big, nor too small')
                return alpha

    return alpha

In [ ]:
def Armijo(f, f_grad, xk, pk):
    # Amijo_Line search
    print('Start line search')
    
    mu1 = 0.01
    tau = 0.1 # 0.5
    
    def phi(alpha):
        return f(xk + alpha * pk)

    def derphi(alpha):
        return np.dot(f_grad(xk + alpha * pk), pk)
    
    alpha_L = 0
    alpha_R = None
    phi_L = phi(0)
    derphi0 = derphi(0)
    derphi_L = derphi(0)
    alphainf= True
    alpha =1 
    phi_min = -1.e2
    eps0 = 1.e-6
    max_iter = 5
    n_iter = 0
    
    if derphi(0) >= 0:
        print(' no descent direction ')
        return None
    

    while n_iter < max_iter:
        print("  {}. alpha between {} and {}".format(n_iter,alpha_L,alpha_R))
        n_iter +=1
        phi_hat= phi(alpha)
        derphi_hat = derphi(alpha)

        if phi_hat > phi(0) + mu1*alpha*derphi(0): #alpha too big # 
            print(" alpha too big: {} > {}".format(phi_hat, phi(0) + mu1*alpha*derphi(0)))

            alphainf= False
            alpha_R = alpha

            delta = alpha_R - alpha_L
            # print('  delta=', delta)
            ## interpolation
            c = (phi_hat - phi_L - derphi_L*delta)/delta**2
            alpha_tilde = alpha_L - derphi_L/(2*c)
            alpha = min(max(alpha_L+ tau*delta, alpha_tilde), alpha_R - tau*delta)
            print('alpha_int=', alpha)
            
        else:
            return alpha

In [ ]:
def make_solver_orig(B, symmetric=False, spd=False):
    """Return a :class:`LinearOperator` that acts as a linear solver for the
    (dense or sparse) square matrix `B`.

    If `B` is symmetric, passing ``symmetric=True`` may try to take advantage of this.
    If `B` is symmetric and positive definite, pass ``spd=True``.
    """
    if spd:
        symmetric = True
# Gauß'sche Eliminationsverfahren - LU Zerlegung (auch LR für left-right)
    if scipy.sparse.issparse(B):
        print('use SuperLU')
            # use SuperLU (unless scipy uses UMFPACK?) -- really slow!
        spLU = scipy.sparse.linalg.splu(B.tocsc(), permc_spec='NATURAL')
        M= scipy.sparse.linalg.LinearOperator(B.shape, dtype=B.dtype, matvec=spLU.solve, matmat=spLU.solve)
        return M
# Cholesky Zerlegung: Matrix muss symmetrisch und positiv definit sein!                  
    else:
        if symmetric:
            print('use Cholesky')
            chol = scipy.linalg.cho_factor(B, check_finite=False)
            solve = lambda x: scipy.linalg.cho_solve(chol, x, check_finite=False)
            return scipy.sparse.linalg.LinearOperator(B.shape, dtype=B.dtype,
                    matvec=solve, matmat=solve)
        else:
            print('Matrix is not symmetric')
            LU = scipy.linalg.lu_factor(B, check_finite=False)
            solve = lambda x: scipy.linalg.lu_solve(LU, x, check_finite=False)
            return scipy.sparse.linalg.LinearOperator(B.shape, dtype=B.dtype,
                    matvec=solve, matmat=solve)

In [ ]:
def make_solver(matrix):
    B= matrix.todense()
    """Return a :class:`LinearOperator` that acts as a linear solver for the
    (dense or sparse) square matrix `B`.

    If `B` is symmetric, passing ``symmetric=True`` may try to take advantage of this.
    If `B` is symmetric and positive definite, pass ``spd=True``.
    """
    
    chol = scipy.linalg.cho_factor(B, check_finite=False) # for s. p.d matrix
    solve = lambda x: scipy.linalg.cho_solve(chol, x, check_finite=False)
    return scipy.sparse.linalg.LinearOperator(B.shape, dtype=B.dtype, matvec=solve, matmat=solve)
       

In [ ]:
def get_defplotPP(u, patches_u, kvs_u):
    """Split solution vector into displacement components."""
    u1 = u[:MP_u.numdofs] 
    u2 = u[MP_u.numdofs:2*MP_u.numdofs]
    
    # restrict solution to each individual patch - BSpline functions
    u1_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u1)
           for p in range(len(patches_u))]
    u2_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u2)
           for p in range(len(patches_u))]

    # evaluate displacement (and "pressure") over a grid in the parameter domain
    # grid variables
    xgrid=np.linspace(0, 1, x_el)
    ygrid=np.linspace(0, 1, y_el)
    xygrid = (xgrid, ygrid)
    #len_xgrid= len(xgrid)
    vrange= None
   
    figsize(7,7)
    fig, ax = plt.subplots()
    ###
    
    max_xval_i = 0
    max_yval_i = 0
    max_rad_i = 0
    max_rad_o = 0
    idx_ri = 0
    idx_ro = 0
    count=0
    # visualization per patch
    for (u1_func, u2_func, (kvs, geo)) in zip(u1_funcs, u2_funcs, patches_u): #u_funcs 
        print('\n patch:', count)
        dis1 = u1_func.grid_eval(xygrid) #x-value
        dis2 = u2_func.grid_eval(xygrid) #y-value
        dis = np.stack((dis1,dis2), axis=-1)
       
        G = geo.grid_eval(xygrid)
        x_val_i=[]
        y_val_i=[]
        x_val_o=[]
        y_val_o=[]
        
        i= y_el-1
    
        radius_inner = np.sqrt( (G[i,0, 0]+dis1[i, 0])**2 + (G[i,0, 1]+dis2[i,0])**2)
        radius_outer = np.sqrt((G[i,y_el-1, 0]+dis1[i, y_el-1])**2 + (G[i,y_el-1, 1]+dis2[i,y_el-1])**2)
        print('inner radius_{}= {}'.format(i, radius_inner))
        print('outer radius_{}= {}'.format(i, radius_outer))
        print('displacement_inner= {}'.format(dis[i, 0,...]))
        print('displacement_outer= {}'.format(dis[i, y_el-1,...]))
        print( 'dis_inner_x/dis_outer_x: ', dis1[i, 0]/dis1[i,y_el-1])
        print( 'dis_inner_y/dis_outer_y: ', dis2[i, 0]/dis2[i, y_el-1])
        
        #print (' ratio:', radius_inner/radius_outer) # with respect to max. inner radius coord.
        
        print (' ratio_inner:', radius_inner/r_in) # with respect to inner radius
        print (' ratio_outer:', radius_outer/r_out) # with respect to outer radius
        
        plot(G[i, 0, 0] + dis1[i, 0], G[i, 0, 1] + dis2[i, 0], 'ro') # inner radius
        plot(G[i, y_el-1, 0] + dis1[i, y_el-1], G[i, y_el-1, 1] + dis2[i, y_el-1], 'bo') # outer radius
        #print(G[ y_el-1, 0, 0] , G[ y_el-1, 0, 1]) # (x, y) patch orientation!
        #print(G[y_el-1, y_el-1, 0], G[y_el-1, y_el-1, 1]) 
        plot(G[y_el-1, y_el-1, 0], G[y_el-1, y_el-1, 1] , 'co') # outer radius


        plot_grid(G[..., 0], G[..., 1], ax=ax, color="lightgrey")
        #plot(G[i, 0, 0] + dis1[i, 0], G[i, 0, 1] + dis2[i, 0], 'mo') # inner radius 
        #plot(G[i, y_el-1, 0] + dis1[i, y_el-1], G[i, y_el-1, 1] + dis2[i, y_el-1], 'co') # outer radius
        
        count+=1
       
        C = np.sqrt(np.power(dis[..., 0], 2) + np.power(dis[..., 1], 2))
        if vrange is None:
            vrange = (C.min(), C.max())
            
        plt.pcolormesh(G[..., 0] + dis[..., 0], G[..., 1] + dis[..., 1], C, shading='gouraud', cmap='viridis', 
                                    vmin=vrange[0], vmax=vrange[1])
        #vis.plot_deformation(dis, ref, geo, ax, vmin=0.0, vmax=1.5e-3)
    colorbar();
    axis('equal')


In [ ]:
#incremental loading
maxload = 2e-1 #1e0 #9e-2 #9e-2# 8e-1 # 6e-2 # 4e-1
nsteps= 3
loading= np.linspace(maxload/nsteps, maxload, nsteps)

In [ ]:
robin_data = [(1, 'right', g_zero, 1)] # instead of Dirichlet-bdc, outer bd fixed

In [ ]:
# solve linearized system #Robin instead of Dirichlet, + Neumann force (inner pressure)
### first iteration ###
XP = MP_block.patch_to_global(0) #p=0
dd = shape(XP)[0]

#initial value
u = np.zeros(dd)
solution= []

# set Neumann boundary force (via incremental loading) 

### Linear elasticity for largest loading
loading_value = loading[-1] # take last loading value
neu_data = [(0,'left',gN), (1,'left',gN), (2,'left',gN), (3,'left',gN)] # set neumann

### first iteration ###
A, b = ass_nonlinsystem_RN(u)
M = ass_mass()
Minv = make_solver_orig(M)
r0 = np.transpose(b).dot(Minv.dot(b)) #L2-norm
print('r0=', r0)

u_d = make_solver_orig(A).dot(b) 
u += u_d

A, b = ass_nonlinsystem_RN(u)
r = np.transpose(b).dot(Minv.dot(b)) #L2-norm
print('residual = ', r)

###-----------------###--------------------###
solution = [u]
# norm of delta u in first step
normed_du0= np.transpose(u_d).dot(Minv.dot(u_d)) #L2-norm
###----------------###---------------------###

# print deformation plot (after first iteration)
get_defplot(u, patches_u, kvs_u)

In [ ]:
get_defplotPP(u, patches_u, kvs_u)

In [ ]:
print(u)

In [ ]:
#incremental loading
maxload = 4e-1 #2e-1 #9e-2# #8e-2# 6e-2 #4e-2#
nsteps= 3
loading= np.linspace(maxload/nsteps, maxload, nsteps)


In [ ]:
robin_data = [(1, 'right', g_zero, 1)] # instead of Dirichlet-bdc, outer bd fixed

In [ ]:
# solve linearized system #Robin instead of Dirichlet, + Neumann force (inner pressure)
### first iteration ###
XP = MP_block.patch_to_global(0) #p=0
dd = shape(XP)[0]

#initial value
u= np.zeros(dd)
solution= []

# set Neumann boundary force (via incremental loading) 

### Linear elasticity for largest loading
loading_value = loading[-1] # take last loading value
neu_data = [(0,'left',gN), (1,'left',gN), (2,'left',gN), (3,'left',gN)] # set neumann/robin data

### first iteration ###
A, b = ass_nonlinsystem_RN(u)
r0 = np.transpose(b).dot(Minv.dot(b)) #L2-norm
print('Residual0 =', r0)
print('Energy0   =', J(u))

#u_d = make_solver_orig(A, symmetric=True, spd=True).dot(b) 
u_d = make_solver_orig(A).dot(b) 
u += u_d

A, b = ass_nonlinsystem_RN(u)
r = np.transpose(b).dot(Minv.dot(b)) #L2-norm
print('Residual  =', r)
print('Energy    =', J(u))

###-----------------###--------------------###
solution = [u]
# norm of delta u in first step
normed_du0= np.transpose(u_d).dot(M.dot(u_d)) #L2-norm
###----------------###---------------------###

# print deformation plot (after first iteration)
get_defplot(u, patches_u, kvs_u)

In [ ]:
get_defplotPP(u, patches_u, kvs_u)

In [ ]:
loading

In [ ]:
u

In [ ]:
# solve linearized variational problem - iterative, without line-search

#initial value
u= np.zeros(dd)

epsilon= 1e-4# 1e-5

solutions = []
stepsizes = [] 
ud_array = []
iter_counts = []


### Linear elasticity for largest loading
loading_value = loading[-1] # take last loading value
neu_data = [(0,'left',gN), (1,'left',gN), (2,'left',gN), (3,'left',gN)] # set neumann data

A, b = ass_nonlinsystem_RN(u)
M = ass_mass()
Minv = make_solver_orig(M)
r0 = np.transpose(b).dot(Minv.dot(b)) #L2-norm
#r0 = np.linalg.norm(b)


print("Norm of rhs for max loading: {}".format(r0))
print("Tolerance:                   {}".format(r0*epsilon))
###

# incremental loading # ----------------------------------
for t in range(len(loading)): # time steps
    print(" \n \n {}. loading: {} \n".format(t+1,loading[t]))
    
    # set Neumann data via incremental loading
    loading_value = loading[t]
    neu_data = [(0,'left',gN), (1,'left',gN), (2,'left',gN), (3,'left',gN)] 

    count = 0
    while True:
        count+=1

        # Assemble matrices and rhs in every iteration step
        if count == 1:
            A, b = ass_nonlinsystem_RN(u)
            r = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm
            #r = np.linalg.norm(b)
            print('Residual =',r)
            print('Energy   =',J(u))
        #else: # Use from last iteration; see below (*)
        #    A, b = ass_nonlinsystem_RN(u)
            
        print(count)

        # # solve system # #
        #u_d = make_solver(A).dot(b) 
        u_d = make_solver_orig(A).dot(b)
        u += u_d            

        # Compute new non-linear residual, already to be used for next iteratio (*)
        A, b = ass_nonlinsystem_RN(u)
        r = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm 
        #r= np.linalg.norm(b)
        print('residual =', r)
        print('energy   =', J(u))
        
        #w1 = np.linalg.inv(A.todense()).dot(b)-u  #TODO: Use solver instead of inverse !!! 
        #w1 = make_solver_orig(A).dot(b)-u         #TODO: We do not want to solve that often!!
        #w2 = b - A.dot(u)
        #err= np.sqrt(np.abs(np.inner(w1,w2)))
        #print('Error=', err)

        #----------------------------------------------------#
        ud_array.append(u_d)
        normed_du = np.transpose(u_d).dot(M.dot(u_d)) #L2-norm
        #normed_du = np.linalg.norm(u_d)/np.sqrt(len(u_d))
        stepsize_du = normed_du # times alpha
        stepsizes.append(stepsize_du)
        #---------------------------------------------------#

        if r < epsilon * r0: # break condition
            break
        elif count == 30:
            break
    #
    ud_array.append(u_d) 
    print('u= ' , u)
    solutions.append(np.array(u))
    iter_counts.append(count)
        
# print deformation plot
get_defplot(u, patches_u, kvs_u)

In [ ]:
get_defplotPP(u, patches_u, kvs_u)

In [ ]:
solutions_iter = solutions

In [ ]:
#solve nonlinear system
#initial value

u= np.zeros(dd)
epsilon= 1e-4# 1e-5

solutions = []
stepsizes = [] 
ud_array = []
iter_counts = []

### Linear elasticity for largest loading
loading_value = loading[-1] # take last loading value
neu_data = [(0,'left',gN), (1,'left',gN), (2,'left',gN), (3,'left',gN)] # set neumann data

A,  b = ass_nonlinsystem_RN(u)
r0 = np.transpose(b).dot(Minv.dot(b)) #L2-norm

print("Norm of rhs for max loading: {}".format(r0))
print("Tolerance:                   {}".format(r0*epsilon))
###

# incremental loading # ----------------------------------
for t in range(len(loading)): # time steps
    print(" \n \n {}. loading: {} \n".format(t+1,loading[t]))
    
    # set Neumann data via incremental loading
    loading_value = loading[t]
    neu_data = [(0,'left',gN), (1,'left',gN), (2,'left',gN), (3,'left',gN)] 
    
    # first residuum
    A, b = ass_nonlinsystem_RN(u)
    r= np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm
    print('Residual=', r)
    

    count= 0
    rejected = False
    
    while True:
        count+=1
        print(count)

        # Assemble matrices and rhs in every iteration step
        A, b = ass_nonlinsystem_RN(u)
        u_d = make_solver_orig(A).dot(b) 
        

        # check if matrix is positive definite in a particular direction u_d
        sp = b.dot(A @ u_d) # For the residual,...
        #if LS.b.dot(u_d) <= 0:
        if sp <= 0 or rejected: 
            print('{}: Matrix is not positive definite (or rejected); use gradient direction anyway!'.format(sp))
            u_d = b
            rejected = False
        else:
            print('{}: Matrix is positive definite; use Newton direction.'.format(sp))
        
        print( 'grad_k*p_k=',  np.dot(grad_J(u), u_d))

        # # do Wolfe-Powell line search # # 
        alpha = Wolfe_LineSearch(J, grad_J, u, u_d) 
        
        # Line Search 
        #alpha= Armijo(J, grad_J, u, u_d) 
        print('alpha=', alpha)
        print(' Energy: ', J(u))
        #print('Gradient: ', grad_J(u))
        #print('Gradient_a: ', grad_a(u))
        
        
        if alpha != None:
            unew = u + alpha * u_d
            # compute new non-linear residuum
            A, b = ass_nonlinsystem_RN(unew)
            rnew = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm
            if rnew > r:
                print("New residual would be {}. Reject this alpha!".format(rnew))
                alpha = None
        
        if alpha == None:           
            print('Use simple line search:')
            #u += 0.5 * u_d # alpha=0.5 (ignore line search)
            # Lets try to do gradient search
            alpha = 1.
            rnew = r+1 ####
            while rnew > r and alpha > 1e-4:
                unew = u + alpha * u_d
                # compute new non-linear residuum
                A, b = ass_nonlinsystem_RN(unew)
                rnew = np.transpose(b).dot(Minv.dot(b)) #dual of L2-norm
                #rnew = np.linalg.norm(LS.b) # compute residuum
                print('  alpha {} gives residual {}'.format(alpha,rnew))
                alpha /= 2
            if rnew > r:
                alpha = None
                print("alpha =", alpha)
                if rejected:
                    break # No way to continue !?
                else:
                    alpha = 0
                    rejected = True
            alpha *= 2 # double alpha
            if alpha >= 1:
                alpha = 2.
                rlast = rnew
                while alpha < 10000 and rnew <= rlast:
                    unew = u + alpha * u_d
                    # compute new non-linear residuum
                    A, b = ass_nonlinsystem_RN(unew)
                    rlast = rnew
                    rnew = np.transpose(b).dot(Minv.dot(b)) #L2-norm
                    print('  alpha {} gives residual {}'.format(alpha,rnew))
                    alpha *= 2 # double alpha
                alpha /= 4 # quater alpha

            print("alpha =", alpha)            
            
        u += alpha * u_d            

            
        # compute new non-linear residuum
        A, b = ass_nonlinsystem_RN(u)
        r = np.transpose(b).dot(Minv.dot(b)) #L2-norm
        print('residual =', r)
        
        #w1 = np.linalg.inv(A.todense()).dot(b)-u
        #w2 = b - A.dot(u)
        #err= np.sqrt(np.abs(np.inner(w1,w2)))
        #print('Error=', err)


        #----------------------------------------------------#
        ud_array.append(u_d)
        normed_du= np.transpose(u_d).dot(M.dot(u_d)) #L2-norm
        stepsize_du = alpha * normed_du # times alpha
        stepsizes.append(stepsize_du)
        #---------------------------------------------------#

        if r < epsilon* r0: # break condition # r < epsilon* r1:
            break
        elif count == 30:
            break
  
    ud_array.append(u_d) 
    solutions.append(np.array(u))
    iter_counts.append(count)
        
# print deformation plot
get_defplot(u, patches_u, kvs_u)

In [ ]:
get_defplotPP(u, patches_u, kvs_u)

In [ ]:
# plot history of step sizes
figsize(6,3)
fig, ax = plt.subplots()
yscale('log')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.plot(stepsizes);
xlabel('steps in delta_u'); ylabel('step size');

In [ ]:
iter_counts

In [ ]:
# ngsolve for maxload = 9e-2 and 3 steps#, (4,4,3) Newton iterations
# 2e-1: (4,4,3) (zero based)
# 4e-1: (5,4,4) #iga(2, 4, 3), [2, 3, 5, 3, 2]
# 5e-1: (5,4,4)
# 6e-1: (5,4,4) 8e-1
# 1e0: (5,4,5)

In [ ]:
figsize(6,3)
fig, ax= plt.subplots()

s_cts= shape(iter_counts)[0]
plot(range(s_cts), iter_counts)

ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.title(' Number of iterations over loading process');
xlabel('Loading'); ylabel('Number of iterations');

In [ ]:
solutions

In [ ]:
solutions_iter

In [ ]:
solutions_iter[0].all == solutions[0]

In [ ]:
solutions_iter[-1].all == solutions[-1]

In [ ]:
"""Visualization functions."""
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.collections import LineCollection


def animate_field(fields, patches_u, kvs_u, vrange=None, res=(50,50),cmap=None, interval=50, progress=False):
    """Animate a sequence of scalar fields over a geometry."""
    fields = list(fields)
    
    figsize(7,7)
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.set_xlim(left=-5.5, right=5)
    ax.set_ylim(bottom=-5, top=5.5)
    
    
    ar= np.linspace(1,3, s_sol)
    factor = ar[0]
    C = None

    if np.isscalar(res):
        res = (res, res)

    # first solution
    #u= LS.complete(fields[0])
    u = fields[0]
    
    #Split solution vector into displacement components
    u1 = u[:MP_u.numdofs] 
    u2 = u[MP_u.numdofs::]
    # restrict solution to each individual patch - BSpline functions
    u1_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u1)
           for p in range(len(patches_u))]
    u2_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u2)
           for p in range(len(patches_u))]

    # evaluate displacement (and "pressure") over a grid in the parameter domain
    # visualization per patch
    for (u1_func, u2_func, (kvs, geo)) in zip(u1_funcs, u2_funcs, patches_u): #u_funcs 
        grd = tuple(np.linspace(s[0], s[1], r) for (s, r) in zip(geo.support, res))
        XY = geo.grid_eval(grd)
        dis1 = u1_func.grid_eval(grd) #x-value
        dis2 = u2_func.grid_eval(grd) #y-value
        dis = np.stack((dis1,dis2), axis=-1)
        if C is None:
            C = np.sqrt(np.power(dis[..., 0], 2) + np.power(dis[..., 1], 2))
        if vrange is None:
            #vrange = (0.0, 1.5e-3)
            vrange = (C.min(), C.max())
        #vis.plot_grid(XY[..., 0] + dis[..., 0], XY[..., 1] + dis[..., 1], ax=ax, color="black")
        quadmesh = plt.pcolormesh(XY[..., 0] + factor* dis[..., 0], XY[..., 1] + factor*dis[..., 1], C, shading='gouraud', cmap='viridis',
                                    vmin=vrange[0], vmax=vrange[1], axes=ax)
        
   
    fig.colorbar(quadmesh, ax=ax);
    tqdm = vis.utils.progress_bar(progress)
    pbar = tqdm(total=len(fields))
    
    def anim_func(i):
        plt.cla()
        ax.set_xlim(left=-5.5, right=5)
        ax.set_ylim(bottom=-5, top=5.5)
        #factor = ar[i] # choose factor for deformation plot
        #u = LS.complete(fields[i])
        u = fields[i]
        u1 = u[:MP_u.numdofs] 
        u2 = u[MP_u.numdofs:]
        
        vrange= None

    # restrict solution to each individual patch - BSpline functions
        u1_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u1)
               for p in range(len(patches_u))]
        u2_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u2)
               for p in range(len(patches_u))]

    # visualization per patch
        for (u1_func, u2_func, (kvs, geo)) in zip(u1_funcs, u2_funcs, patches_u): #u_funcs 
            grd = tuple(np.linspace(s[0], s[1], r) for (s, r) in zip(geo.support, res))
            XY = geo.grid_eval(grd)
            dis1 = u1_func.grid_eval(grd) #x-value
            dis2 = u2_func.grid_eval(grd) #y-value
            dis = np.stack((dis1,dis2), axis=-1)
            C = np.sqrt(np.power(dis[..., 0], 2) + np.power(dis[..., 1], 2))
            if vrange is None:
                #vrange = (0.0, 1.5e-3)
                vrange = (C.min(), C.max())
            quadmesh = plt.pcolormesh(XY[..., 0] + factor*dis[..., 0], XY[..., 1] + factor*dis[..., 1], C, shading='gouraud', cmap='viridis', 
                                      vmin=vrange[0], vmax=vrange[1], axes=ax)
        
            #quadmesh.set_array(C.ravel())
        pbar.update()
        if i == len(u) - 1:
            pbar.close()
  
    return animation.FuncAnimation(plt.gcf(), anim_func, frames=len(fields), interval=interval, repeat=False)


In [ ]:
from IPython.display import HTML

figsize(14, 4)
s_sol= shape(solutions)[0]
fields = [solutions[tt] for tt in range(s_sol)]
HTML(animate_field(fields, patches_u, kvs_u, res=(50,70), interval=335, progress=True).to_html5_video())

In [ ]:
plt.clf()

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.collections import LineCollection

plt.rcParams['animation.html'] = 'html5'
plt.rcParams['font.size'] = 18

fig, ax = plt.subplots(figsize=(8,8))
ax.set_aspect('equal')
res=(50,70)
#vrange=(0.0, 1.5e-3)

s_sol= shape(solutions)[0]
ims = []
ar= np.linspace(1,2, s_sol)
factor = 1


#fields = list(ud_array)
fields = list(solutions)

for tt in range(s_sol):
    #u = LS.complete(fields[tt])
    u = fields[tt]
    factor = ar[tt] # choose factor for deformation plot
    vrange = None
    ims_q = []
    u1 = u[:MP_u.numdofs] 
    u2 = u[MP_u.numdofs:]

    # restrict solution to each individual patch - BSpline functions
    u1_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u1)
            for p in range(len(patches_u))]
    u2_funcs = [geometry.BSplineFunc(kvs_u, MP_u.global_to_patch(p) @ u2)
            for p in range(len(patches_u))]
    plt.ioff()
    
    # visualization per patch
    for (u1_func, u2_func, (kvs, geo)) in zip(u1_funcs, u2_funcs, patches_u): #u_funcs 
        im_q=[]
        grd = tuple(np.linspace(s[0], s[1], r) for (s, r) in zip(geo.support, res))
        XY = geo.grid_eval(grd)
        dis1 = u1_func.grid_eval(grd) #x-value
        dis2 = u2_func.grid_eval(grd) #y-value
        dis = np.stack((dis1,dis2), axis=-1)
        C = np.sqrt(np.power(dis[..., 0], 2) + np.power(dis[..., 1], 2))
        if vrange is None:
            #vrange = (0.0, 1.5e-3)
            vrange = (C.min(), C.max())
        im_q = plt.pcolormesh(XY[..., 0] + factor*dis[..., 0], XY[..., 1] + factor*dis[..., 1], C, shading='gouraud', cmap='viridis', 
                                    vmin=vrange[0], vmax=vrange[1], axes=ax) # shading for smoothing
        
        im_q.set_array(C.ravel()) 
        ims_q.append(im_q) 
    
    ims.append(ims_q)
    #print(shape(ims))
#print(ims[0])
    
#fig.colorbar(im_q, ax=ax);
colorbar();
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat= False) # repeat_delay=2000



#ani.to_html5_video()


Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'))
#ani.save("rtest.mp4", writer=writer )
vid = ani.to_html5_video()

plt.close()



In [ ]:
ani

In [ ]:
 #solve nonlinear system
#initial value

u= np.zeros(dd)

epsilon= 1e-4# 1e-5

solutions = []
stepsizes = [] 
ud_array = []
iter_counts = []


### Linear elasticity for largest loading
loading_value = loading[-1] # take last loading value
neu_data = [(0,'left',gN), (1,'left',gN), (2,'left',gN), (3,'left',gN)] # set neumann data


A, b = ass_nonlinsystem_RN(u)
r0= np.transpose(b).dot(Minv.dot(b)) #Dual of L2-norm

print("Norm of rhs for max loading: {}".format(r0))
print("Tolerance:                   {}".format(r0*epsilon))
###



# incremental loading # ----------------------------------
for t in range(len(loading)): # time steps
    print(" \n \n {}. loading: {} \n".format(t+1,loading[t]))
    
    # set Neumann data via incremental loading
    loading_value = loading[t]
    neu_data = [(0,'left',gN), (1,'left',gN), (2,'left',gN), (3,'left',gN)] 
    
    # first residuum
    A, b = ass_nonlinsystem_RN(u)
    r= np.transpose(b).dot(Minv.dot(b)) #Dual of L2-norm
    print('Residual=', r)

    count= 0
    rejected = False
    
    while True:
        count+=1
        print(count)

        # Assemble matrices and rhs in every iteration step
        A, b = ass_nonlinsystem_RN(u)
        u_d = make_solver_orig(A).dot(b) 
        

        # check if matrix is positive definite in a particular direction u_d
        sp = b.dot(A @ u_d) # For the residual,...
        #if LS.b.dot(u_d) <= 0:
        if sp <= 0 or rejected: 
            print('{}: Matrix is not positive definite (or rejected); use gradient direction anyway!'.format(sp))
            u_d = b
            rejected = False
        else:
            print('{}: Matrix is positive definite; use Newton direction.'.format(sp))
        
        print( 'grad_k*p_k=',  np.dot(grad_J(u), u_d))

        # # do Wolfe-Powell line search # # 
        alpha= Wolfe_LineSearch(J, grad_J, u, u_d) 
        
        # Line Search 
        #alpha= Armijo(J, grad_J, u, u_d) 
        print('alpha=', alpha)
        
        if alpha == None:           
            print('Use simple line search:')
            #u += 0.5 * u_d # alpha=0.5 (ignore line search)
            # Lets try to do gradient search
            alpha = 1.
            rnew = r+1 ####
            while rnew > r and alpha > 1e-4:
                unew = u + alpha * u_d
                # compute new non-linear residuum
                A, b = ass_nonlinsystem_RN(unew)
                rnew = np.transpose(b).dot(Minv.dot(b)) #Dual of L2-norm
                #rnew = np.linalg.norm(LS.b) # compute residuum
                print('  alpha {} gives residual {}'.format(alpha,rnew))
                alpha /= 2
            if rnew > r:
                alpha = None
                print("alpha =", alpha)
                if rejected:
                    break # No way to continue !?
                else:
                    alpha = 0
                    rejected = True
            alpha *= 2 # double alpha
            if alpha >= 1:
                alpha = 2.
                rlast = rnew
                while alpha < 10000 and rnew <= rlast:
                    unew = u + alpha * u_d
                    # compute new non-linear residuum
                    A, b = ass_nonlinsystem_RN(unew)
                    rlast = rnew
                    rnew = np.transpose(b).dot(Minv.dot(b)) #Dual of L2-norm
                    print('  alpha {} gives residual {}'.format(alpha,rnew))
                    alpha *= 2 # double alpha
                alpha /= 4 # quater alpha

            print("alpha =", alpha)            
            
        u += alpha * u_d            

            
        # compute new non-linear residuum
        A, b = ass_nonlinsystem_RN(u)
        r = np.transpose(b).dot(Minv.dot(b)) #Dual of L2-norm
        print('residual =', r)


        #----------------------------------------------------#
        ud_array.append(u_d)
        normed_du= np.transpose(u_d).dot(M.dot(u_d)) #L2-norm
        stepsize_du = alpha * normed_du # times alpha
        stepsizes.append(stepsize_du)
        #---------------------------------------------------#

        if r < epsilon* r0: # break condition # r < epsilon* r1:
            break
        elif count == 30:
            break
  
    ud_array.append(u_d) 
    solutions.append(np.array(u))
    iter_counts.append(count)
        
# print deformation plot
get_defplot(u, patches_u, kvs_u)

In [ ]:
# plot history of step sizes
figsize(6,3)
fig, ax = plt.subplots()
yscale('log')

ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.plot(stepsizes);
xlabel('steps in delta_u'); ylabel('step size');

In [ ]:
figsize(6,3)
fig, ax= plt.subplots()

s_cts= shape(iter_counts)[0]
plot(range(s_cts), iter_counts)
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.title(' Number of iterations over loading process');
xlabel('Loading'); ylabel('Number of iterations');

In [ ]:
# Line search based on energy


u= np.zeros(dd)

epsilon= 1e-4# 1e-5

solutions = []
stepsizes = [] 
ud_array = []
iter_counts = []


### Linear elasticity for largest loading
loading_value = loading[-1] # take last loading value
neu_data = [(0,'left',gN), (1,'left',gN), (2,'left',gN), (3,'left',gN)] # set neumann data


A, b = ass_nonlinsystem_RN(u)
r0= np.transpose(b).dot(Minv.dot(b)) #Dual of L2-norm

print("Norm of rhs for max loading: {}".format(r0))
print("Tolerance:                   {}".format(r0*epsilon))
###



J0 = 0

# incremental loading # ----------------------------------
for t in range(len(loading)): # time steps
    print(" \n \n {}. loading: {} \n".format(t+1,loading[t]))
    
    # set Neumann data via incremental loading
    loading_value = loading[t]
    neu_data = [(0,'left',gN), (1,'left',gN), (2,'left',gN), (3,'left',gN)] 
    
    # first residuum
    A, b = ass_nonlinsystem_RN(u)
    r = np.transpose(b).dot(Minv.dot(b)) #Dual of L2-norm
    print('Residual=', r)

    count= 0
    rejected = False
    
    while True:
        count+=1
        print(count)

        # Assemble matrices and rhs in every iteration step
        A, b = ass_nonlinsystem_RN(u)
        u_d = make_solver_orig(A).dot(b) 
        
        alpha = 1.
        J1 = J(u+alpha*u_d)
        print("alpha = {} gives energy {}".format(alpha, J1))
        downfactor = .7
        upfactor   = 1.3
        
        while J1>J0:
            alpha *= downfactor
            J1 = J(u+alpha*u_d)
            print("alpha = {} gives energy {}".format(alpha, J1))
        if alpha == 1.:
            J2 = J(u+upfactor*alpha*u_d)
            print("alpha = {} gives energy {}".format(upfactor*alpha, J2))
            while J2<J1:
                alpha *= upfactor
                J1 = J2
                J2 = J(u+upfactor*alpha*u_d)
                print("alpha = {} gives energy {}".format(upfactor*alpha, J2))
        print("alpha = {}".format(alpha))
        u += alpha * u_d       
            
        # compute new non-linear residuum
        A, b = ass_nonlinsystem_RN(u)
        r = np.transpose(b).dot(Minv.dot(b)) #Dual of L2-norm
        print('residual = {}'.format(r))
        J0 = J(u)
        print('energy   = {}'.format(J0))


        #----------------------------------------------------#
        ud_array.append(u_d)
        normed_du= np.transpose(u_d).dot(M.dot(u_d)) #L2-norm
        stepsize_du = alpha * normed_du # times alpha
        stepsizes.append(stepsize_du)
        #---------------------------------------------------#

        if r < epsilon* r0: # break condition # r < epsilon* r1:
            break
        elif count == 30:
            break
  
    ud_array.append(u_d) 
    solutions.append(np.array(u))
    iter_counts.append(count)
        
# print deformation plot
get_defplot(u, patches_u, kvs_u)
print("done")